In [6]:
import numpy as np
import cv2
import glob
from icecream import ic
from pickle import dump, load
from copy import deepcopy

Checkerboard and criteria for calibration

In [2]:
CHECKERBOARD = (10,7)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

Get calibration images

In [9]:
# Open the default camera
cam = cv2.VideoCapture(0)

cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280 )
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cam.get(cv2.CAP_PROP_FPS))

ic(frame_width, frame_height, fps)

n = 0

while True:
    ret, raw = cam.read()

    # Display the captured frame
    gray = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)
    frame = deepcopy(raw)
    
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    if ret:
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)

        # Draw and display the corners
        frame = cv2.drawChessboardCorners(frame, CHECKERBOARD, corners2, ret)
       
    cv2.imshow('img',frame)
    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break
    elif cv2.waitKey(1) == ord('c'):
        if ret:
            cv2.imwrite('calibration images/{}.png'.format(n), raw)
            ic(n)
            n += 1

# Release the capture and writer objects
cam.release()
cv2.destroyAllWindows()

ic| frame_width: 1280, frame_height: 720, fps: 30
ic| n: 0
ic| n: 1
ic| n: 2
ic| n: 3
ic| n: 4
ic| n: 5
ic| n: 6
ic| n: 7
ic| n: 8
ic| n: 9
ic| n: 10
ic| n: 11
ic| n: 12
ic| n: 13
ic| n: 14
ic| n: 15
ic| n: 16
ic| n: 17
ic| n: 18
ic| n: 19
ic| n: 20
ic| n: 21
ic| n: 22
ic| n: 23
ic| n: 24
ic| n: 25
ic| n: 26
ic| n: 27
ic| n: 28
ic| n: 29
ic| n: 30
ic| n: 31
ic| n: 32
ic| n: 33
ic| n: 34
ic| n: 35
ic| n: 36
ic| n: 37
ic| n: 38
ic| n: 39
ic| n: 40
ic| n: 41
ic| n: 42
ic| n: 43
ic| n: 44
ic| n: 45
ic| n: 46
ic| n: 47
ic| n: 48


Calibration

In [10]:
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= 24
prev_img_shape = None
 
# Extracting path of individual image stored in a given directory
images = glob.glob('./calibration images/*.png')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
    cv2.imshow('img',img)
    cv2.waitKey(1)
 
cv2.destroyAllWindows()
 
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""

'\nPerforming camera calibration by \npassing the value of known 3D points (objpoints)\nand corresponding pixel coordinates of the \ndetected corners (imgpoints)\n'

In [11]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
ic(mtx, dist)

with open('calibration_logi.calib', 'wb') as f:
    obj = (mtx,dist)
    dump(obj, f)

ic| mtx: array([[1.42442011e+03, 0.00000000e+00, 6.00936662e+02],
                [0.00000000e+00, 1.41755565e+03, 3.80159596e+02],
                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
    dist: array([[ 0.01854327, -0.04146034, -0.00178674,  0.00135992, -0.42533342]])
